<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/veo_api_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 3.5 MB/s eta 0:00:00


In [2]:
#import the genai library
from google import genai

In [3]:
#step2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
#step2: Vertex AI: read the api key from the user data

#for vertex, you need to have a service account or login/auth
#login as the current user for colab testing purposes
#from google.colab import auth
#auth.authenticate_user()

#then configure the client with the project and location settings

#Uncomment the following if you are running on Vertex setup

#using the colab userdata
#from google.colab import userdata
#client = genai.Client(vertexai=True, #ensure to turn on VertexAI
#                      project=userdata.get("GOOGLE_CLOUD_PROJECT"),
#                      location=userdata.get("GOOGLE_CLOUD_REGION")) # e.g., 'us-central1'

#or use the folliwing if you want to read from environment keys
#import os
#client = genai.Client(vertexai=True,
#                      project=os.environ["GOOGLE_CLOUD_PROJECT"],
#                      location=os.environ["GOOGLE_CLOUD_REGION"]) # e.g., 'us-central1'

In [6]:
#step 3: Get model details
model_name = "models/veo-2.0-generate-001"
try:
    model_details = client.models.get(model=model_name) #
    print(f"Details for model '{model_name}':")
    print(f"Model Name: {model_details.name}")
    print(f"Input Token Limit: {model_details.input_token_limit}")
    print(f"Output Token Limit: {model_details.output_token_limit}")
except Exception as e:
    print(f"Error retrieving model details for '{model_name}': {e}")

Details for model 'models/veo-2.0-generate-001':
Model Name: models/veo-2.0-generate-001
Input Token Limit: 480
Output Token Limit: 8192


In [ ]:
#step 4: Kick off the operation

#ensure types import to support Video generation config
from google.genai import types

# Define the prompt for Veo 2
video_prompt = "A cinematic timelapse of clouds moving over a mountain range at sunset."

# Kick off the video generation operation
operation = client.models.generate_videos(
    model=model_name,
    prompt=video_prompt,
    config=types.GenerateVideosConfig(
        person_generation="dont_allow",  # "dont_allow" or "allow_adult"
        aspect_ratio="16:9",  # "16:9" or "9:16"
    ),
)



In [16]:
print(f"Video generation started. Operation name: {operation.name}")


Video generation started. Operation name: models/veo-2.0-generate-001/operations/0b7r3hnsv6c0


In [17]:
#step 5: Poll the status of the operaiton to see if it is complete
import time
while not operation.done:
    time.sleep(20)
    operation = client.operations.get(operation)

In [19]:
#step 6: Once the operation is complete, download the video
for n, generated_video in enumerate(operation.response.generated_videos):
    client.files.download(file=generated_video.video)
    generated_video.video.save(f"video{n}.mp4")  # save the video

In [21]:
#step 7: Play the video
from IPython.display import Video

Video("video0.mp4", embed=True)